In [6]:
import pandas as pd

ds_path = 'data/rio_listings.csv'

df = pd.read_csv(ds_path, sep = ',')#, encoding='latin-1')
df.describe()

,id,scrape_id,description,host_id,host_listings_count,host_total_listings_count,neighbourhood_group_cleansed,latitude,longitude,accommodates,...,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,license,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
count,3.600800e+04,3.600800e+04,0.0,3.600800e+04,35999.000000,35999.000000,0.0,36008.000000,36008.000000,36008.000000,...,25721.000000,25723.000000,25722.000000,25722.000000,0.0,36008.000000,36008.000000,36008.000000,36008.000000,25706.000000
mean,4.445730e+17,2.023123e+13,NaN,1.784179e+08,17.221728,28.521237,NaN,-22.967580,-43.249377,4.071956,...,4.881911,4.868024,4.858699,4.701961,NaN,9.493224,8.647634,0.724228,0.114613,1.046277
std,4.418582e+17,0.000000e+00,NaN,1.769684e+08,95.150438,141.409181,NaN,0.034687,0.099241,2.358069,...,0.320784,0.340892,0.322370,0.442303,NaN,25.677082,25.412461,1.889419,0.985128,1.157816
min,1.787800e+04,2.023123e+13,NaN,1.671000e+03,1.000000,1.000000,NaN,-23.073276,-43.723009,1.000000,...,0.000000,0.000000,0.000000,0.000000,NaN,1.000000,0.000000,0.000000,0.000000,0.010000
25%,2.863794e+07,2.023123e+13,NaN,2.416786e+07,1.000000,1.000000,NaN,-22.984712,-43.306566,2.000000,...,4.890000,4.870000,4.850000,4.620000,NaN,1.000000,1.000000,0.000000,0.000000,0.200000
50%,5.761368e+17,2.023123e+13,NaN,9.644573e+07,2.000000,3.000000,NaN,-22.972860,-43.194825,4.000000,...,5.000000,5.000000,4.970000,4.820000,NaN,2.000000,1.000000,0.000000,0.000000,0.670000
75%,8.726761e+17,2.023123e+13,NaN,3.256093e+08,6.000000,8.000000,NaN,-22.956277,-43.185573,5.000000,...,5.000000,5.000000,5.000000,5.000000,NaN,5.000000,3.000000,1.000000,0.000000,1.500000
max,1.053823e+18,2.023123e+13,NaN,5.527145e+08,1405.000000,1927.000000,NaN,-22.749690,-43.104400,16.000000,...,5.000000,5.000000,5.000000,5.000000,NaN,185.000000,185.000000,21.000000,19.000000,13.750000


In [7]:
clustering_df = df.loc[:, ['review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness', 'review_scores_checkin', 'review_scores_communication', 'review_scores_location','review_scores_value']].dropna()

In [8]:
from math import ceil, floor

classification_df = df.loc[:, ['room_type', 'bathrooms_text', 'latitude', 'longitude', 'accommodates', 'beds', 'price']].dropna()

def extract_bathrooms(bathroom_text):
    shared_baths = 0
    half_baths = 0
    full_baths = 0
    try:
        baths_num = float(''.join([c for c in bathroom_text if not c.isalpha()]).strip())
    except:
        baths_num = 1

    if 'hared' in bathroom_text: shared_baths += ceil(baths_num)
    if 'alf' in bathroom_text: half_baths += floor(baths_num)
    
    if baths_num != floor(baths_num):
        half_baths += 1
    
    full_baths += floor(baths_num)
    
    return full_baths, half_baths,shared_baths

classification_df['bathrooms'] = [extract_bathrooms(text)[0] for text in classification_df['bathrooms_text']]
classification_df['half_bathrooms'] = [extract_bathrooms(text)[1] for text in classification_df['bathrooms_text']]
classification_df['shared_bathrooms'] = [extract_bathrooms(text)[2] for text in classification_df['bathrooms_text']]
classification_df['price'] = [float(s[1:].replace(',','')) for s in classification_df['price']]


classification_df = classification_df.drop('bathrooms_text', axis = 1)

classification_df['room_type'] = classification_df['room_type'].apply(lambda text: text.replace(' ', '_'))

classification_df.head()

,room_type,latitude,longitude,accommodates,beds,price,bathrooms,half_bathrooms,shared_bathrooms
0,Entire_home/apt,-22.96599,-43.17940,5,2.0,1357.0,1,0,0
1,Entire_home/apt,-22.97735,-43.19105,3,1.0,865.0,1,0,0
2,Entire_home/apt,-22.98107,-43.19136,2,1.0,373.0,1,1,0
4,Entire_home/apt,-22.96825,-43.18237,4,1.0,366.0,1,0,0
5,Entire_home/apt,-22.97696,-43.18933,5,3.0,368.0,2,0,0


In [9]:
itemset_df = df.loc[:, ['host_response_time', 'host_response_rate', 'host_acceptance_rate', 'host_is_superhost', 'host_verifications', 'host_has_profile_pic', 'host_identity_verified']].dropna()

itemset_df['host_is_superhost'] = itemset_df['host_is_superhost'] == 't'
itemset_df['host_has_profile_pic'] = itemset_df['host_has_profile_pic'] == 't'
itemset_df['host_identity_verified'] = itemset_df['host_identity_verified'] == 't'

itemset_df[f'host_has_email_verification'] = itemset_df['host_verifications'].apply(lambda list_text: 'email' in list(list_text))
itemset_df[f'host_has_phone_verification'] = itemset_df['host_verifications'].apply(lambda list_text: 'phone' in list(list_text))
itemset_df[f'host_has_work_email_verification'] = itemset_df['host_verifications'].apply(lambda list_text: 'work_email' in list(list_text))
itemset_df[f'host_has_photographer_verification'] = itemset_df['host_verifications'].apply(lambda list_text: 'photographer' in list(list_text))

itemset_df['host_response_time'] = itemset_df['host_response_time'].apply(lambda text: text.replace(' ', '_'))
itemset_df = pd.get_dummies(itemset_df, columns = ['host_response_time'])

itemset_df['host_response_rate'] = itemset_df['host_response_rate'].apply(lambda val_text: int(val_text[:-1]))
itemset_df['host_acceptance_rate'] = itemset_df['host_acceptance_rate'].apply(lambda val_text: int(val_text[:-1]))

itemset_df = itemset_df.drop(['host_verifications'], axis = 1)

itemset_df.head()



,host_response_rate,host_acceptance_rate,host_is_superhost,host_has_profile_pic,host_identity_verified,host_has_email_verification,host_has_phone_verification,host_has_work_email_verification,host_has_photographer_verification,host_response_time_a_few_days_or_more,host_response_time_within_a_day,host_response_time_within_a_few_hours,host_response_time_within_an_hour
0,100,96,True,True,True,False,False,False,False,False,False,False,True
1,100,80,True,True,True,False,False,False,False,False,False,False,True
2,100,98,True,True,True,False,False,False,False,False,False,False,True
3,90,89,False,True,True,False,False,False,False,False,False,False,True
4,100,93,False,True,False,False,False,False,False,False,False,True,False


In [10]:
clustering_df.to_csv('data/clustering.csv', index = False)
classification_df.to_csv('data/classification.csv', index = False)
itemset_df.to_csv('data/itemset.csv', index = False)